In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import hdbscan
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
from sklearn.metrics.cluster import contingency_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 

# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

df = X.copy()
df['y'] = y
 
df=df.dropna()

df['target'] = df['y'].map({'<=50K': 0, '<=50K.': 0, '>50K': 1, '>50K.': 1})
df = df.drop('y', axis=1)

df_num = df[['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week', 'target']]

# Function to sample 1000 rows from each group
def sample_from_group(group):
    return group.sample(min(len(group), 5000), random_state=42)

# Apply the function to each group defined by the unique values in the 'target' column
sampled_df = df_num.groupby('target', group_keys=False).apply(sample_from_group)

# Reset index
sampled_df.reset_index(drop=True, inplace=True)

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit and transform the scaler on your numerical DataFrame
df_num_scaled = scaler.fit_transform(sampled_df)

# Convert the scaled array back to a DataFrame
df_num = pd.DataFrame(df_num_scaled, columns=df_num.columns)

# Count occurrences of each unique value in the 'target' column
target_counts = df_num['target'].value_counts()

# Print the counts
print(target_counts)

num_observations = df_num.shape[0]
print("Number of Observations:", num_observations)

0.0    5000
1.0    5000
Name: target, dtype: int64
Number of Observations: 10000


In [5]:
X = df_num.drop('target', axis=1)
y = df_num['target']

In [23]:
# Function to create a new DataFrame with unique values and their counts
def create_unique_count_df(column_name):
    if column_name not in X.columns:
        print(f"Column '{column_name}' not found in the DataFrame.")
        return pd.DataFrame()  # Return an empty DataFrame
    unique_values = X[column_name].unique()
    value_counts = X[column_name].value_counts()
    unique_count_df = pd.DataFrame({'Value': unique_values, 'Count': value_counts})
    return unique_count_df

# Get the column names
columns = X.columns

# Dictionary to store unique values and counts DataFrames for each column
unique_values_counts_dict = {}

# Iterate over each column, create a new DataFrame, and store it in the dictionary
for col in columns:
    unique_count_df = create_unique_count_df(col)
    unique_values_counts_dict[col] = unique_count_df

# Save each DataFrame for each column to separate files
for col, df_count in unique_values_counts_dict.items():
    csv_file_name = f"{col}_unique_values_counts.csv"
    df_count.to_csv(csv_file_name, index=False)

In [24]:
file_path = 'Hours-per-week_unique_values_counts.csv'  
df = pd.read_csv(file_path)
# Create DataFrame
df = pd.DataFrame(df)

# Define bins for the intervals
bins = [0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00]

# Create intervals and sum 'Count' values within each interval
df['Interval'] = pd.cut(df['Value'], bins=bins, include_lowest=False)
result_df = df.groupby('Interval')['Count'].sum().reset_index()

print(result_df)

result_df.to_csv("hours_all.csv", index=True)

       Interval  Count
0   (0.0, 0.05]    355
1   (0.05, 0.1]    468
2   (0.1, 0.15]    190
3   (0.15, 0.2]    657
4   (0.2, 0.25]    133
5   (0.25, 0.3]    742
6   (0.3, 0.35]    277
7   (0.35, 0.4]   1317
8   (0.4, 0.45]    484
9   (0.45, 0.5]    239
10  (0.5, 0.55]     36
11  (0.55, 0.6]   4510
12  (0.6, 0.65]    371
13  (0.65, 0.7]     36
14  (0.7, 0.75]     76
15  (0.75, 0.8]     15
16  (0.8, 0.85]     23
17  (0.85, 0.9]     11
18  (0.9, 0.95]      4
19  (0.95, 1.0]     45


In [38]:
list = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']

for l in list:
    print(l)
    #file_path = 'age_unique_values_counts.csv'  
    df = X[l].copy()
    # Create DataFrame
    df = pd.DataFrame(df)

    # Define bins for the intervals
    bins = [0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00]

    # Create intervals and sum 'Count' values within each interval
    df['Interval'] = pd.cut(df[l], bins=bins, include_lowest=True)

    # Group by 'Interval' and count the number of occurrences
    bin_counts = df.groupby('Interval').size().reset_index(name='Count')

    # Sort bins by interval
    bin_counts['Interval'] = bin_counts['Interval'].astype(str)
    bin_counts = bin_counts.sort_values(by='Interval')

    print(bin_counts)

    ordered_df = df.groupby('Interval')
    print(ordered_df)

age
          Interval  Count
0   (-0.001, 0.05]    432
1      (0.05, 0.1]    659
2      (0.1, 0.15]    577
3      (0.15, 0.2]    955
4      (0.2, 0.25]   1121
5      (0.25, 0.3]    905
6      (0.3, 0.35]   1138
7      (0.35, 0.4]   1106
8      (0.4, 0.45]    699
9      (0.45, 0.5]    828
10     (0.5, 0.55]    562
11     (0.55, 0.6]    346
12     (0.6, 0.65]    316
13     (0.65, 0.7]    173
14     (0.7, 0.75]     79
15     (0.75, 0.8]     54
16     (0.8, 0.85]     23
17     (0.85, 0.9]     12
18     (0.9, 0.95]      1
19     (0.95, 1.0]     14
fnlwgt
          Interval  Count
0   (-0.001, 0.05]   1134
1      (0.05, 0.1]   2005
2      (0.1, 0.15]   2784
3      (0.15, 0.2]   2009
4      (0.2, 0.25]    905
5      (0.25, 0.3]    622
6      (0.3, 0.35]    310
7      (0.35, 0.4]    111
8      (0.4, 0.45]     50
9      (0.45, 0.5]     39
10     (0.5, 0.55]     12
11     (0.55, 0.6]      6
12     (0.6, 0.65]      9
13     (0.65, 0.7]      2
14     (0.7, 0.75]      0
15     (0.75, 0.8]      0
1

In [13]:
import pandas as pd
from sklearn.cluster import KMeans


number_clusters = [3, 4, 5, 6, 7, 8, 9]

for n in number_clusters:
    kmeans = KMeans(n_clusters=n, random_state=42)
    cluster_labels = kmeans.fit_predict(X)

    # Create a new DataFrame with the cluster labels
    X_n = df_num.copy()
    X_n['cluster'] = cluster_labels

    print(f"Number of clusters: {n}")

    for cluster in range(n):
        cluster_data = X_n[X_n['cluster'] == cluster]
        total_in_cluster = len(cluster_data)
        class_0_in_cluster = len(cluster_data[cluster_data['target'] == 0])
        class_1_in_cluster = len(cluster_data[cluster_data['target'] == 1])
        
        # Calculate purity score for the current cluster
        if total_in_cluster > 0:
            majority_class = cluster_data['target'].mode()[0]
            correct_predictions = len(cluster_data[cluster_data['target'] == majority_class])
            cluster_purity_score = correct_predictions / total_in_cluster
        else:
            cluster_purity_score = 0

        print(f"Cluster {cluster}:")
        print(f"  Total observations: {total_in_cluster}")
        print(f"  Observations in class 0: {class_0_in_cluster}")
        print(f"  Observations in class 1: {class_1_in_cluster}")
        print(f"  Purity score: {cluster_purity_score:.2f}")

    # Calculate overall purity score
    purity_scores = []
    for cluster in range(n):
        cluster_data = X_n[X_n['cluster'] == cluster]
        if len(cluster_data) > 0:
            majority_class = cluster_data['target'].mode()[0]
            correct_predictions = len(cluster_data[cluster_data['target'] == majority_class])
            purity_scores.append(correct_predictions / len(cluster_data))
    
    overall_purity_score = sum(purity_scores) / n
    print(f"Overall purity score: {overall_purity_score:.2f}\n")


Number of clusters: 3
Cluster 0:
  Total observations: 609
  Observations in class 0: 137
  Observations in class 1: 472
  Purity score: 0.78
Cluster 1:
  Total observations: 5173
  Observations in class 0: 3300
  Observations in class 1: 1873
  Purity score: 0.64
Cluster 2:
  Total observations: 4218
  Observations in class 0: 1563
  Observations in class 1: 2655
  Purity score: 0.63
Overall purity score: 0.68

Number of clusters: 4
Cluster 0:
  Total observations: 609
  Observations in class 0: 137
  Observations in class 1: 472
  Purity score: 0.78
Cluster 1:
  Total observations: 5147
  Observations in class 0: 3295
  Observations in class 1: 1852
  Purity score: 0.64
Cluster 2:
  Total observations: 4129
  Observations in class 0: 1568
  Observations in class 1: 2561
  Purity score: 0.62
Cluster 3:
  Total observations: 115
  Observations in class 0: 0
  Observations in class 1: 115
  Purity score: 1.00
Overall purity score: 0.76

Number of clusters: 5
Cluster 0:
  Total observati

In [29]:
true_labels = y

def calculate_purity_and_sizes(cluster_labels, true_labels):
    cluster_purities = []
    cluster_sizes = []
    class_counts = []
    for cluster_label in np.unique(cluster_labels):
        cluster_indices = np.where(cluster_labels == cluster_label)[0]
        cluster_X = X.iloc[cluster_indices]
        cluster_labels_true = true_labels.iloc[cluster_indices]
        label_counts = np.bincount(cluster_labels_true)
        majority_label_count = np.max(label_counts)
        cluster_purity = majority_label_count / len(cluster_labels_true)
        cluster_purities.append(cluster_purity)
        cluster_sizes.append(len(cluster_indices))
        class_counts.append(label_counts)
    overall_purity = np.mean(cluster_purities)
    return overall_purity, cluster_purities, cluster_sizes, class_counts

# Perform KMeans clustering for 3, 4, and 5 clusters
cluster_numbers = [3, 4, 5, 6, 7, 8, 9]
overall_purities = []
cluster_purities_iterations = []
cluster_sizes_iterations = []
class_counts_iterations = []

for n_clusters in cluster_numbers:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(X)

    # Calculate purity, sizes, and class counts for each cluster
    overall_purity, cluster_purities, cluster_sizes, class_counts = calculate_purity_and_sizes(cluster_labels, true_labels)
    overall_purities.append(overall_purity)
    cluster_purities_iterations.append(cluster_purities)
    cluster_sizes_iterations.append(cluster_sizes)
    class_counts_iterations.append(class_counts)

for i, n_clusters in enumerate(cluster_numbers):
    print(f"Overall Purity for {n_clusters} clusters: {overall_purities[i]:.2f}")
    for cluster_label, (purity, size, class_count) in enumerate(zip(cluster_purities_iterations[i], cluster_sizes_iterations[i], class_counts_iterations[i])):
        class_0_count = class_count[0] if len(class_count) > 0 else 0
        class_1_count = class_count[1] if len(class_count) > 1 else 0
        print(f"{cluster_label}, {purity:.2f}, {size}, {class_0_count}/{class_1_count}")

Overall Purity for 3 clusters: 0.68
0, 0.78, 609, 137/472
1, 0.64, 5173, 3300/1873
2, 0.63, 4218, 1563/2655
Overall Purity for 4 clusters: 0.76
0, 0.78, 609, 137/472
1, 0.64, 5147, 3295/1852
2, 0.62, 4129, 1568/2561
3, 1.00, 115, 0/115
Overall Purity for 5 clusters: 0.75
0, 0.78, 608, 136/472
1, 0.62, 3944, 1518/2426
2, 0.58, 2461, 1042/1419
3, 1.00, 115, 0/115
4, 0.80, 2872, 2304/568
Overall Purity for 6 clusters: 0.74
0, 0.78, 608, 136/472
1, 0.57, 3571, 1552/2019
2, 0.58, 2150, 903/1247
3, 1.00, 115, 0/115
4, 0.82, 2516, 2065/451
5, 0.67, 1040, 344/696
Overall Purity for 7 clusters: 0.71
0, 0.78, 608, 136/472
1, 0.57, 2980, 1282/1698
2, 0.58, 2006, 836/1170
3, 1.00, 115, 0/115
4, 0.51, 1023, 521/502
5, 0.67, 991, 326/665
6, 0.83, 2277, 1899/378
Overall Purity for 8 clusters: 0.72
0, 0.78, 604, 132/472
1, 0.68, 2662, 1797/865
2, 0.54, 1329, 613/716
3, 0.61, 2756, 1086/1670
4, 0.95, 648, 614/34
5, 0.72, 924, 261/663
6, 1.00, 115, 0/115
7, 0.52, 962, 497/465
Overall Purity for 9 cluste

In [30]:
kmeans = KMeans(n_clusters=9, random_state=42)
cluster_labels = kmeans.fit_predict(X)

X_9 = X.copy()
print(X)
X_9['cluster'] = cluster_labels
print(X_9)

           age    fnlwgt  capital-gain  capital-loss  hours-per-week
0     0.246575  0.020062       0.03103      0.000000        0.551020
1     0.301370  0.254985       0.00000      0.000000        0.397959
2     0.068493  0.353789       0.00000      0.000000        0.397959
3     0.465753  0.144167       0.00000      0.000000        0.295918
4     0.000000  0.075273       0.01055      0.000000        0.193878
...        ...       ...           ...           ...             ...
9995  0.191781  0.103344       0.00000      0.000000        0.397959
9996  0.465753  0.313124       0.00000      0.000000        0.071429
9997  0.150685  0.044391       0.00000      0.424654        0.500000
9998  0.356164  0.143599       0.00000      0.000000        0.346939
9999  0.575342  0.145647       0.00000      0.000000        0.346939

[10000 rows x 5 columns]
           age    fnlwgt  capital-gain  capital-loss  hours-per-week  cluster
0     0.246575  0.020062       0.03103      0.000000        0.551020

In [31]:
cluster_2_df = X_9[X_9['cluster'] == 2][['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]
cluster_7_df = X_9[X_9['cluster'] == 7][['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]

In [52]:
list = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']

for l in list:
    print(l)
    #file_path = 'age_unique_values_counts.csv'  
    df = cluster_2_df[l].copy()
    # Create DataFrame
    df = pd.DataFrame(df)

    # Define bins for the intervals
    bins = [0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00]

    # Create intervals and sum 'Count' values within each interval
    df['Interval'] = pd.cut(df[l], bins=bins, include_lowest=True)

    # Group by 'Interval' and count the number of occurrences
    bin_counts = df.groupby('Interval').size().reset_index(name='Count')

    # Sort bins by interval
    bin_counts['Interval'] = bin_counts['Interval'].astype(str)
    bin_counts = bin_counts.sort_values(by='Interval')

    count_list = bin_counts['Count'].tolist()
    print(count_list)

    ordered_df = df.groupby('Interval')
    #print(ordered_df)

age
[0, 0, 0, 0, 0, 0, 0, 0, 9, 67, 309, 240, 243, 146, 65, 48, 22, 11, 1, 13]
fnlwgt
[121, 232, 351, 239, 118, 56, 32, 13, 7, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
capital-gain
[1041, 59, 23, 31, 18, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
capital-loss
[1172, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
hours-per-week
[26, 43, 42, 76, 63, 57, 75, 686, 77, 28, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
# Function to create a new DataFrame with unique values and their counts
def create_unique_count_df(column_name):
    if column_name not in cluster_2_df.columns:
        print(f"Column '{column_name}' not found in the DataFrame.")
        return pd.DataFrame()  # Return an empty DataFrame
    unique_values = cluster_2_df[column_name].unique()
    value_counts = cluster_2_df[column_name].value_counts()
    unique_count_df = pd.DataFrame({'Value': unique_values, 'Count': value_counts})
    return unique_count_df

# Get the column names
columns = cluster_2_df.columns

# Dictionary to store unique values and counts DataFrames for each column
unique_values_counts_dict = {}

# Iterate over each column, create a new DataFrame, and store it in the dictionary
for col in columns:
    unique_count_df = create_unique_count_df(col)
    unique_values_counts_dict[col] = unique_count_df

# Save each DataFrame for each column to separate files
for col, df_count in unique_values_counts_dict.items():
    csv_file_name = f"{col}_unique_values_counts.csv"
    df_count.to_csv(csv_file_name, index=True)

In [27]:
file_path = 'hours-per-week_unique_values_counts.csv'  
df = pd.read_csv(file_path)
# Create DataFrame
df = pd.DataFrame(df)

# Define bins for the intervals
bins = [0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00]

# Create intervals and sum 'Count' values within each interval
df['Interval'] = pd.cut(df['Value'], bins=bins, include_lowest=False)
result_df = df.groupby('Interval')['Count'].sum()#.reset_index()

print(result_df)

result_df.to_csv("hours_bins.csv", index=True)

Interval
(0.0, 0.05]    720
(0.05, 0.1]     40
(0.1, 0.15]    122
(0.15, 0.2]     74
(0.2, 0.25]     18
(0.25, 0.3]     26
(0.3, 0.35]     29
(0.35, 0.4]     99
(0.4, 0.45]     31
(0.45, 0.5]     12
(0.5, 0.55]      1
(0.55, 0.6]      0
(0.6, 0.65]      0
(0.65, 0.7]      0
(0.7, 0.75]      0
(0.75, 0.8]      0
(0.8, 0.85]      0
(0.85, 0.9]      0
(0.9, 0.95]      0
(0.95, 1.0]      0
Name: Count, dtype: int64
